In [4]:
import sys
import sklearn
import numpy as np
import os

np.random.seed(42)

%matplotlib inline
import matplotlib as mpl
import matplotlib.pyplot as plt
mpl.rc('axes', labelsize=14)
mpl.rc('xtick', labelsize=12)
mpl.rc('ytick', labelsize=12)

8번 - 여러종류의 분류기 훈련시키고 투표기반 앙상블방법을 사용해 테스트세트로 확인

In [5]:
from sklearn.datasets import fetch_openml
mnist = fetch_openml('mnist_784', version=1, as_frame=False)
mnist.keys()

dict_keys(['data', 'target', 'frame', 'feature_names', 'target_names', 'DESCR', 'details', 'categories', 'url'])

In [6]:
X, y = mnist["data"], mnist["target"]

In [7]:
from sklearn.model_selection import train_test_split

X_train1, X_test, y_train1, y_test = train_test_split(X, y, test_size=10000, random_state=42)
X_train,X_val,y_train,y_val=train_test_split(X_train1, y_train1, test_size=10000, random_state=42)

In [8]:
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
rf_clf=RandomForestClassifier(n_estimators=5,random_state=42)
et_clf=ExtraTreesClassifier(n_estimators=5,random_state=42)
svm_clf=SVC(kernel="rbf",random_state=42)
dt_clf=DecisionTreeClassifier(max_depth=5,random_state=42)

rf_clf.fit(X_train,y_train)
et_clf.fit(X_train,y_train)
svm_clf.fit(X_train,y_train)
dt_clf.fit(X_train,y_train)

DecisionTreeClassifier(ccp_alpha=0.0, class_weight=None, criterion='gini',
                       max_depth=5, max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort='deprecated',
                       random_state=42, splitter='best')

In [9]:
[model.score(X_val,y_val) for model in [rf_clf,et_clf,svm_clf,dt_clf]]

[0.9182, 0.9225, 0.9788, 0.679]

In [12]:
from sklearn.ensemble import VotingClassifier

vote_clf1=VotingClassifier(
    estimators=[('rf',rf_clf),('et',et_clf),('svm',svm_clf),('dt',dt_clf)],
    voting='hard'
)
vote_clf2=VotingClassifier(
    estimators=[('rf',rf_clf),('et',et_clf),('dt',dt_clf)],
    voting='soft'
)


vote_clf1.fit(X_train,y_train)
vote_clf2.fit(X_train,y_train)

VotingClassifier(estimators=[('rf',
                              RandomForestClassifier(bootstrap=True,
                                                     ccp_alpha=0.0,
                                                     class_weight=None,
                                                     criterion='gini',
                                                     max_depth=None,
                                                     max_features='auto',
                                                     max_leaf_nodes=None,
                                                     max_samples=None,
                                                     min_impurity_decrease=0.0,
                                                     min_impurity_split=None,
                                                     min_samples_leaf=1,
                                                     min_samples_split=2,
                                                     min_weight_fraction_leaf=0.0,
        

In [14]:
vote_clf1.score(X_val,y_val)

0.9516

In [15]:
vote_clf2.score(X_val,y_val)

0.9313

In [16]:
vote_clf1.score(X_test,y_test)

0.9502

In [17]:
vote_clf2.score(X_test,y_test)

0.9271

9번 - 스태킹

In [23]:
X_val_predictions = np.empty((len(X_val), 4), dtype=np.float32)

for index, estimator in enumerate([rf_clf,et_clf,svm_clf,dt_clf]):
    X_val_predictions[:, index] = estimator.predict(X_val)

In [24]:
rnd_forest_blender = RandomForestClassifier(n_estimators=200, oob_score=True, random_state=42)
rnd_forest_blender.fit(X_val_predictions, y_val)

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=200,
                       n_jobs=None, oob_score=True, random_state=42, verbose=0,
                       warm_start=False)

In [25]:
rnd_forest_blender.oob_score_

0.9743

In [27]:
X_test_predictions = np.empty((len(X_test), 4), dtype=np.float32)

for index, estimator in enumerate([rf_clf,et_clf,svm_clf,dt_clf]):
    X_test_predictions[:, index] = estimator.predict(X_test)

In [28]:
y_pred = rnd_forest_blender.predict(X_test_predictions)

In [29]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test, y_pred)

0.9704